In [184]:
import boto3
import torch
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
import boto3
import datetime
import time
from time import strftime,gmtime
import json
import os
import urllib
import sys
import io

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
sns_client = boto3.client('sns')
region = boto_session.region_name
bucket = sm_session.default_bucket()
prefix = 'monai-async-inference-brain-tumor'
role = "arn:aws:iam::617011600974:role/service-role/AmazonSageMaker-ExecutionRole-20210316T155032"

print(region)
print(role)
print(bucket)
print(prefix)









eu-west-1
arn:aws:iam::617011600974:role/service-role/AmazonSageMaker-ExecutionRole-20210316T155032
sagemaker-eu-west-1-617011600974
monai-async-inference-brain-tumor


In [185]:
#model_name = "MONAI-DEMO-MultiModelModel" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
#model_url = "s3://sagemaker-eu-west-1-617011600974/monai-on-sagemaker-docker-2022-06-03-08-24-03-985/output/model.tar.gz"

!mkdir model_and_code
!cp ./model/model.pth model_and_code/
#torch.save(model, 'model_and_code/model.pth')
!mkdir model_and_code/code
!cp ./source/* model_and_code/code
!tar cvzf model.tar.gz -C model_and_code/ . 

from sagemaker.s3 import S3Uploader
file_key = 'model.tar.gz'
model_artifact = S3Uploader.upload(file_key,'s3://{}/{}/model'.format(bucket, prefix))
print(model_artifact)




mkdir: cannot create directory ‘model_and_code’: File exists
mkdir: cannot create directory ‘model_and_code/code’: File exists
cp: omitting directory ‘./source/__pycache__’
./
./model.pth
./code/
./code/inference.py
./code/requirements.txt
./code/__init__.py
s3://sagemaker-eu-west-1-617011600974/monai-async-inference-brain-tumor/model/model.tar.gz


In [186]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.g4dn.xlarge'
pytorch_inference_image_uri = retrieve('pytorch',
                                       region,
                                       version='1.11.0',
                                       py_version='py38',
                                       instance_type = deploy_instance_type,
                                       accelerator_type=None,
                                       image_scope='inference')

# pytorch_inference_image_uri = "763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.11.0-gpu-py38-cu113-ubuntu20.04-sagemaker"
print(pytorch_inference_image_uri)



763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.11.0-gpu-py38


In [187]:


container = pytorch_inference_image_uri
model_name = 'sagemaker-monai-brain-tumor-{0}'.format(str(int(time.time())))
print(container)
print(model_name)


763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.11.0-gpu-py38
sagemaker-monai-brain-tumor-1654808393


In [188]:
create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_artifact,
        'Environment': {
            'TS_MAX_REQUEST_SIZE': '200000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '200000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }
    },    
)

In [189]:
print(model_name)
endpoint_config_name = f"monaiEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1
        }
    ]
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

sagemaker-monai-brain-tumor-1654808393
Created EndpointConfig: arn:aws:sagemaker:eu-west-1:617011600974:endpoint-config/monaiendpointconfig-2022-06-09-20-59-58


In [190]:
bucket_prefix = "monai-async-inference"
resource_name = "Monai-AsyncInferenceDemo-SNS"


In [191]:
response = sns_client.create_topic(Name="Monai-Async-Demo-ErrorTopic")
error_topic= response['TopicArn']
print(error_topic)

arn:aws:sns:eu-west-1:617011600974:Monai-Async-Demo-ErrorTopic


In [192]:
response = sns_client.create_topic(Name="Monai-Async-Demo-SuccessTopic")
success_topic = response['TopicArn']
print(success_topic)


arn:aws:sns:eu-west-1:617011600974:Monai-Async-Demo-SuccessTopic


In [193]:
response = sns_client.list_topics()
topics = response["Topics"]
print(topics)

[{'TopicArn': 'arn:aws:sns:eu-west-1:617011600974:AmazonRekognitionECGAlert'}, {'TopicArn': 'arn:aws:sns:eu-west-1:617011600974:BCDataBrewAtena'}, {'TopicArn': 'arn:aws:sns:eu-west-1:617011600974:Monai-Async-Demo-ErrorTopic'}, {'TopicArn': 'arn:aws:sns:eu-west-1:617011600974:Monai-Async-Demo-SuccessTopic'}]


In [ ]:
#Note: Replace with your email id

# email_id = 'your-email@domain-name.com'
# email_sub_1 = sns_client.subscribe(
#     TopicArn=success_topic,
#     Protocol='email',
#     Endpoint=email_id)

# email_sub_2 = sns_client.subscribe(
#     TopicArn=error_topic,
#     Protocol='email',
#     Endpoint=email_id)

#Note: You will need to confirm by clicking on the email you recieve to complete the subscription

In [194]:
print(model_name)
endpoint_config_name = f"Monai-AsyncEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket}/{bucket_prefix}/output",
            #  Optionally specify Amazon SNS topics
            "NotificationConfig": {
              "SuccessTopic": success_topic,
              "ErrorTopic": error_topic,
            }
        },
        "ClientConfig": {
            "MaxConcurrentInvocationsPerInstance": 2
        }
    }
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

sagemaker-monai-brain-tumor-1654808393
Created EndpointConfig: arn:aws:sagemaker:eu-west-1:617011600974:endpoint-config/monai-asyncendpointconfig-2022-06-09-21-00-52


In [195]:
endpoint_name = f"monai-async-sm-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
create_endpoint_response = sm_client.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
print(f"Creating Endpoint: {create_endpoint_response['EndpointArn']}")

Creating Endpoint: arn:aws:sagemaker:eu-west-1:617011600974:endpoint/monai-async-sm-2022-06-09-21-00-56


In [196]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

Waiting for endpoint to create...
Endpoint Status: InService


In [197]:
s3_uri = "s3://bcinspectio/old/brain_tumor/Task01_BrainTumor/imagesTr/BRATS_001.nii.gz"
response = sm_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=s3_uri)
output_location = response['OutputLocation']
print(f"OutputLocation: {output_location}")



OutputLocation: s3://sagemaker-eu-west-1-617011600974/monai-async-inference/output/9420a136-3d68-4794-84be-5be4b16542c4.out


In [34]:
from botocore.exceptions import ClientError

def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sm_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response['Error']['Code'] == 'NoSuchKey':
                print("waiting for output...")
                time.sleep(2)
                continue
            raise


In [35]:


output = get_output(output_location)
print(f"Output size in bytes: {((sys.getsizeof(output)))}")



waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting fo

KeyboardInterrupt: 